### Datasets

### https://drive.google.com/drive/folders/1X6WF-rgXLaI4Wn8KoOHlb9W8aSUKzmhm

### Imports

In [246]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [247]:
data=pd.read_csv('mobile_recommendation_system_dataset.csv')

In [248]:
data

,name,ratings,price,imgURL,corpus
0,"REDMI Note 12 Pro 5G (Onyx Black, 128 GB)",4.2,23999,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM6 SystemAndroid 12Processor T...
1,"OPPO F11 Pro (Aurora Green, 128 GB)",4.5,"₹20,999",https://rukminim2.flixcart.com/image/312/312/k...,Storage128 GBRAM6 GBExpandable Storage256GB S...
2,"REDMI Note 11 (Starburst White, 64 GB)",4.2,13149,https://rukminim2.flixcart.com/image/312/312/x...,Storage64 GBRAM4 SystemAndroid 11Processor Sp...
3,"OnePlus Nord CE 5G (Blue Void, 256 GB)",4.1,21999,https://rukminim2.flixcart.com/image/312/312/x...,Storage256 GBRAM12 SystemAndroid Q 11Processo...
4,"APPLE iPhone 13 mini (Blue, 128 GB)",4.6,3537,https://rukminim2.flixcart.com/image/312/312/k...,Storage128 SystemiOS 15Processor TypeA15 Bion...
...,...,...,...,...,...
2541,"REDMI Note 9 (Shadow Black, 64 GB)",4.3,11999,https://rukminim2.flixcart.com/image/312/312/k...,Storage64 GBRAM4 GBExpandable Storage512GB Sy...
2542,"OnePlus 9 5G (Astral Black, 128 GB)",3.9,30203,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM8 SystemAndroid 11Processor S...
2543,Motorola a50,4.1,1549,https://rukminim2.flixcart.com/image/312/312/k...,NaN
2544,"SAMSUNG Galaxy S22 Ultra 5G (Phantom Black, 25...",4.3,20463,https://rukminim2.flixcart.com/image/312/312/x...,Storage256 GBRAM12 SystemAndroid 12Processor ...


### Cleaning

In [249]:
data.isnull().sum()

name        0
ratings     0
price       0
imgURL      0
corpus     12
dtype: int64

In [250]:
data=data.dropna()

In [251]:
data.duplicated().sum()

1

In [252]:
data=data.drop_duplicates()

### Normalizing the Ratings and Price

In [253]:
scaler=MinMaxScaler()
data[['ratings']]=scaler.fit_transform(data[['ratings']])

### Using Collabarative Filtering

In [254]:
user_id_column_names = ['user_id', 'user_id_', 'user_id ', 'User ID', 'User ID_', 'UserID', 'UserID_']
user_id_column = next((col for col in user_id_column_names if col in data.columns), None)

if user_id_column:
    data.rename(columns={user_id_column: 'userId'}, inplace=True)
else:
    data['userId'] = range(1, len(data) + 1)

data['userId'] = pd.to_numeric(data['userId'], errors='coerce').fillna(0).astype(int)

In [255]:
data

,name,ratings,price,imgURL,corpus,userId
0,"REDMI Note 12 Pro 5G (Onyx Black, 128 GB)",0.619048,23999,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM6 SystemAndroid 12Processor T...,1
1,"OPPO F11 Pro (Aurora Green, 128 GB)",0.761905,"₹20,999",https://rukminim2.flixcart.com/image/312/312/k...,Storage128 GBRAM6 GBExpandable Storage256GB S...,2
2,"REDMI Note 11 (Starburst White, 64 GB)",0.619048,13149,https://rukminim2.flixcart.com/image/312/312/x...,Storage64 GBRAM4 SystemAndroid 11Processor Sp...,3
3,"OnePlus Nord CE 5G (Blue Void, 256 GB)",0.571429,21999,https://rukminim2.flixcart.com/image/312/312/x...,Storage256 GBRAM12 SystemAndroid Q 11Processo...,4
4,"APPLE iPhone 13 mini (Blue, 128 GB)",0.809524,3537,https://rukminim2.flixcart.com/image/312/312/k...,Storage128 SystemiOS 15Processor TypeA15 Bion...,5
...,...,...,...,...,...,...
2540,"SAMSUNG Galaxy S20 FE 5G (Cloud Navy, 128 GB)",0.619048,27440,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM8 SystemAndroid 10Processor S...,2529
2541,"REDMI Note 9 (Shadow Black, 64 GB)",0.666667,11999,https://rukminim2.flixcart.com/image/312/312/k...,Storage64 GBRAM4 GBExpandable Storage512GB Sy...,2530
2542,"OnePlus 9 5G (Astral Black, 128 GB)",0.476190,30203,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM8 SystemAndroid 11Processor S...,2531
2544,"SAMSUNG Galaxy S22 Ultra 5G (Phantom Black, 25...",0.666667,20463,https://rukminim2.flixcart.com/image/312/312/x...,Storage256 GBRAM12 SystemAndroid 12Processor ...,2532


In [256]:
user_ratings=data.pivot_table(index='userId',columns='name',values='ratings')
user_ratings.fillna(0,inplace=True)

In [257]:
user_similarity=cosine_similarity(user_ratings)
user_similarity_df=pd.DataFrame(user_similarity,index=user_ratings.index,columns=user_ratings.index)

In [258]:
def get_recommendation(user_id, n_recommendations=10):
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).index[1:]
    user_ratings_avg = user_ratings.mean(axis=0)
    
    recommendations = pd.Series(dtype=float)
    for user in similar_users:
        user_rated_items = user_ratings.loc[user]
        recommended_items = user_rated_items[user_rated_items > 0].index
        recommended_scores = user_ratings_avg[recommended_items].sort_values(ascending=False)
        recommendations = pd.concat([recommendations, recommended_scores])
    
    return recommendations.groupby(recommendations.index).mean().sort_values(ascending=False).head(n_recommendations)

### Using Content Based Filtering

In [260]:
tfidf=TfidfVectorizer(stop_words='english')
tfidf_matrix=tfidf.fit_transform(data['corpus'])

In [261]:
cosine_sim=cosine_similarity(tfidf_matrix,tfidf_matrix)

In [262]:
def get_content_based_recommendations(name, cosine_sim=cosine_sim):
    if name not in data['name'].values:
        print(f"Item '{name}' not found in Dataset.")
        return []
    
    idx = data.index[data['name'] == name][0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    product_indices = [i[0] for i in sim_scores]
    return data['name'].iloc[product_indices]

### Results

In [263]:
user_id = 1
collab_recommendations = get_recommendation(user_id)
print("Collaborative Filtering Recommendations for user", user_id)
print(collab_recommendations)

Collaborative Filtering Recommendations for user 1
OPPO Reno10 5G (Ice Blue, 256 GB)             0.003516
OPPO Reno10 5G (Silvery Grey, 256 GB)         0.001861
SAMSUNG Metro 313                             0.001767
OnePlus Nord 2T 5G (Gray Shadow, 128 GB)      0.001410
OnePlus Nord CE 2 5G (Gray Mirror, 128 GB)    0.001278
OnePlus 10R 5G (Sierra Black, 256 GB)         0.001241
SAMSUNG Galaxy A12 (Blue, 128 GB)             0.001128
SAMSUNG Guru Music 2                          0.001053
OnePlus Nord 2 5G (Gray Sierra, 128 GB)       0.001034
OnePlus Nord CE 2 5G (Bahama Blue, 128 GB)    0.001034
dtype: float64


In [264]:
def main():
    product_name = input("Enter the product name: ")
    matching_products = data[data['name'].str.contains(product_name, case=False, na=False)]

    if matching_products.empty:
        print("No matching products found.")
        return
    
    print("Matching Products:")
    for idx, row in matching_products.iterrows():
        print(f"{row.name}. {row['name']}")
    
    product_choice = int(input("Select a product by number: "))
    
    if product_choice not in matching_products.index:
        print("Invalid choice. Showing default recommendations.")
        selected_product_name = matching_products['name'].iloc[0]
    else:
        selected_product_name = matching_products.loc[product_choice, 'name']

    print(f"\nSelected Product name and number: '{selected_product_name}' & '{product_choice}'")
    print("\nChoose recommendation method:")
    print("1. Collaborative Filtering")
    print("2. Content-Based Filtering")
    method_choice = int(input("Enter choice (1 or 2): "))
    
    if method_choice == 1:
        user_id = matching_products.index.get_loc(product_choice) + 1
        recommendations = get_recommendation(user_id)
        if not recommendations.empty:
            print("\nCollaborative Filtering Recommendations:\n")
            for idx, rec in enumerate(recommendations.index):
                print(f"SL No: {data[data['name'] == rec].index[0]}, Product Name: {rec}")
        else:
            print("\nNo collaborative recommendations available.")

    elif method_choice == 2:
        recommendations = get_content_based_recommendations(selected_product_name)
        if not recommendations.empty:
            print(f"\nContent-Based Recommendations for '{selected_product_name}':\n")
            for idx, rec in enumerate(recommendations):
                print(f"SL No: {data[data['name'] == rec].index[0]}, Product Name: {rec}")
        else:
            print("\nNo content-based recommendations available.")
    else:
        print("\nInvalid choice.")

if __name__ == "__main__":
    main()

Matching Products:
4. APPLE iPhone 13 mini (Blue, 128 GB)
5. APPLE iPhone 6s Plus (Gold, 64 GB)
16. APPLE iPhone 14 Plus ((PRODUCT)RED, 256 GB)
40. APPLE iPhone XR ((PRODUCT)RED, 64 GB)
43. APPLE iPhone 14 Pro (Gold, 256 GB)
54. APPLE iPhone 14 Plus (Midnight, 256 GB)
72. APPLE iPhone 6s (Silver, 64 GB)
75. APPLE iPhone 14 Pro Max (Gold, 512 GB)
78. APPLE iPhone 13 Pro Max (Sierra Blue, 1 TB)
79. APPLE iPhone 14 Pro (Gold, 128 GB)
89. APPLE iPhone 12 (Blue, 128 GB)
95. APPLE iPhone SE (Red, 128 GB)
102. APPLE iPhone SE (White, 128 GB)
109. Apple iPhone XR (Black, 128 GB) (Includes EarPods, Power Adapter)
112. APPLE iPhone 6s Plus (Rose Gold, 128 GB)
118. Apple iPhone XR (Black, 64 GB) (Includes EarPods, Power Adapter)
121. APPLE iPhone XS (Silver, 256 GB)
122. APPLE iPhone 5C (Blue, 16 GB)
131. APPLE iPhone SE 3rd Gen (Starlight, 128 GB)
132. APPLE iPhone XS Max (Silver, 64 GB)
141. APPLE iPhone 7 Plus (Silver, 256 GB)
164. APPLE iPhone 14 Pro Max (Silver, 256 GB)
171. APPLE iPhone SE 